In [1]:
from qtpg.team import Team
from qtpg.agent import Agent
from qtpg.trainer import Trainer
from qtpg.figure13 import Figure13
import numpy as np

In [2]:
# gp params
numLearners = 4
numAgents = 1
numRuns = 1
numGens = 100
numEpisodes = 50
gap = 0.3
# rl params
alpha, discount, epsilon = 0.9, 0.9, 0.1
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0#-0.01
goalReached = 100

In [ ]:
# search tests
max_rules = 5

trainer = Trainer(gap, numLearners, numAgents, max_rules, alpha, discount, epsilon)
trainer.createInitAgents()
env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
env.memoryRepeat = 0.0
for agent in trainer.agents:
    for gen in range(numGens):
        selected_rule = agent.team.select_rule()
        offspring = agent.team.search(selected_rule, env)
        agent.team.evaluate_rule(offspring)
    env.reset()
    print(agent.team.rule_pool)

In [3]:
for run in range(numRuns):
    # init agent population
    trainer = Trainer(gap, numLearners, numAgents, alpha, discount, epsilon)
    
    # init env
    env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
    env.memoryRepeat = 0.0
    # search space to make rules
    trainer.generateRules(env)
    env.memoryRepeat = -0.01    
    trainer.createInitAgents()

    champs = []
    for gen in range(numGens):
        print(gen)
        env.reset()
        for agent in trainer.agents:
            print(agent.win)
            env.reset()
            episode_fitness = [] # new
            for ep in range(numEpisodes):
                if agent.win is not True:
                    env.reset()
                    fitness, win, states, seq = agent.evaluate_fitness(env)
                    episode_fitness.append(fitness)
#                     print(agent.team.q_table)
                    if win == True:
                        print('win stuff start')
                        print(states)
                        agent.win = True
                        for q_value in agent.team.q_table:
                            print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))
                        for i in range(5):
                            env.reset()
                            agent.team.victory_lap(env, seq)
                        print('\n\n\n')
                        for q_value in agent.team.q_table:
                            print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))
                        env.reset()
#                         agent.team.epsilon = 0.0
                        score, win, states = agent.replay(env)
                        print(agent.team.q_table)
                        print('win stuff end!')
                        print('REPLAY')
                        print(score)
                        print(win)
                        if not win:
                            agent.win = False
                        else:
                            champs.append(agent)
                            print('win replay!')
                        
                        print(states)
                        print('!!!!')
            # average fitness over episodes...
            average_fitness = 0
            average_fitness = np.mean(episode_fitness)
            print(average_fitness)
            agent.fitness = average_fitness
        trainer.evolve()

d25cef5c-4fd2-470e-aace-7835b53fb59c
[1, 0, 2, 0]
(0, 1)
7afba89e-e0a1-4eb5-b62b-f4a37af85472
[0, 1, 3, -1]
(3, 1)
d83bffac-5867-47d7-a258-0ed4fb9ddfaf
[1, 2, 1, 3]
(0, 1)
65b83b81-9efc-4358-8d1f-ac9a8cd63040
[0, 4, 2, 3]
(2, 1)
2fc83574-6a06-4e6a-a6e9-f0c632a80c99
[1, 4, 2, 4]
(1, 1)
5
5
start!
[1, 0, 2, 0]
[0, 1, 3, -1]
[1, 2, 1, 3]
[0, 4, 2, 3]
[1, 4, 2, 4]
end!
0
False
0.10659999999999953
False
0.15499999999999958
False
0.15059999999999957
False
0.13079999999999958
False
0.1901999999999995
False
0.13739999999999963
False
0.14619999999999952
False
0.09339999999999948
False
0.08459999999999958
False
0.07359999999999958
False
0.1373999999999995
False
0.08899999999999957
False
0.08019999999999956
False
0.1307999999999996
False
0.09999999999999963
False
0.06259999999999956
False
0.16599999999999956
False
0.11099999999999953
False
0.18579999999999952
False
0.2737999999999995
1
False
0.17259999999999953
False
0.13519999999999952
False
0.016399999999999536
False
0.22319999999999954
False
0

KeyboardInterrupt: 

In [ ]:
print(len(champs))
champ_index = 0
champs[champ_index].team.epsilon = 0.0

for q_value in champs[champ_index].team.q_table:
    print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))

wins = 0
for i in range(100):
    env.reset()
    score, win, states = champs[champ_index].replay(env)
    print(win)
    if win:
        print(states)
        wins += 1
    print(score)
print(wins)